In [2]:
import sqlite3
import pandas as pd

sql1 = '''
SELECT d.doc_label as year,
       word_id,
       word_str,
       sum(word_count) as word_count
  FROM docword w
       JOIN
       doc d USING (
           doc_id
       )
 GROUP BY doc_label,word_id'''

sql2 = '''
SELECT d.doc_label as year,
       sum(word_count) as word_count
  FROM docword w
       JOIN
       doc d USING (
           doc_id
       )
 GROUP BY doc_label'''

db = sqlite3.connect('../battelle.db')
DW = pd.read_sql_query(sql1,db)
WCY = pd.read_sql_query(sql2,db) # Word counts per year

In [3]:
DW['word_count_adj'] = 0 # Add a new column and set a default value

In [4]:
for i, year in enumerate(WCY['year']):
    wc = WCY[WCY['year'] == year]['word_count'][i]
    DW.loc[DW['year'] == year,'word_count_adj'] = (DW.loc[DW['year'] == year,'word_count'] / wc) * 100

In [10]:
'''
# Checking for years in which a word count does not exist (meaning = 0)
x = 0
max = 10000
for i, year in enumerate(WCY['year']):
    for word_str in DW['word_str']:
        if DW.loc[(DW['year'] == year) & (DW['word_str'] == word_str)].empty:
            print('No {} for {}'.format(word_str,year))
        x = x + 1 
        if x == max: break
    if x == max: break
'''

In [4]:
sql3 = '''
SELECT word_str,
       sum(word_count) AS n
  FROM topicword_long
 GROUP BY word_str
 ORDER BY n DESC
 LIMIT 1000'''
TW = pd.read_sql_query(sql3,db)
words = [word for word in TW['word_str']]

In [8]:
#%matplotlib inline
import matplotlib
import matplotlib.style
matplotlib.style.use('ggplot')

t = {}
for word in words:
    try:
        t[word] += 1
    except:
        t[word] = 1
        try:
            ax = DW[['year','word_count_adj']][DW['word_str']==word].plot(x='year', kind='bar', legend=False,title=word)
            fig = ax.get_figure()
            fig.savefig('word-plots/{}.png'.format(word))
            fig.clear()
        except:
            print(word,"not found")
        

dna not found
elsevier not found
rna not found
pcr not found
ieee not found
mirnas not found
b.v not found
mrna not found
mir not found
american not found
mirna not found
cdna not found
hcc not found
springer-verlag not found
bsa not found
degc not found
rt-pcr not found
g/l not found
springer not found
mg/l not found
hsp not found
kda not found
snps not found
wiley not found
nps not found
rnas not found


/Users/rca2t/anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
